In [1]:
import pandas as pd
from gensim import corpora, models
from gensim.parsing.preprocessing import preprocess_documents
from bs4 import BeautifulSoup
from glob import glob
import os
import numpy as np

## Pandas stuff

In [32]:
df = pd.read_csv('financial_services_rules.csv', usecols=
                 ['document_number', 'publication_date'])
# df['publication_date']= pd.to_datetime(df['publication_date'])
df = df.sort_values(by='publication_date')
print(df.shape)
df.reset_index(inplace=True, drop=True)
df.head(5)

(3630, 7)


,title,type,abstract,document_number,publication_date,agencies,excerpts
0,Bank Holding Companies and Change in Bank Control,Proposed Rule,The Board of Governors of the Federal Reserve ...,01-43,2001-01-03,"Federal Reserve System, Treasury Department",The Board of Governors of the Federal Reserve ...
1,Regulation Y; Docket Nos. R-1057 and R-1062; B...,Rule,The Board of Governors of the Federal Reserve ...,01-34,2001-01-03,Federal Reserve System,The Board of Governors of the Federal Reserve ...
2,Bank Holding Companies and Change in Bank Control,Rule,The Board of Governors of the Federal Reserve ...,01-42,2001-01-03,"Federal Reserve System, Treasury Department",The Board of Governors of the Federal Reserve ...
3,Activities and Investments of Insured State Banks,Rule,The FDIC is adopting a final rule to implement...,01-175,2001-01-05,Federal Deposit Insurance Corporation,The FDIC is adopting a final rule to implement...
4,Delegation of Authority to Disclose and Reques...,Rule,The Commodity Futures Trading Commission (Comm...,01-595,2001-01-09,Commodity Futures Trading Commission,The Commodity Futures Trading Commission (Comm...


In [35]:
# Save
id2date = dict(zip(df.document_number, df.publication_date))
np.save('id2date.npy', id2date) 

In [36]:
id2date

{'01-43': '2001-01-03',
 '01-34': '2001-01-03',
 '01-42': '2001-01-03',
 '01-175': '2001-01-05',
 '01-595': '2001-01-09',
 '01-3': '2001-01-10',
 '01-784': '2001-01-11',
 '01-536': '2001-01-16',
 '01-1073': '2001-01-25',
 '01-1614': '2001-01-30',
 '01-1305': '2001-01-31',
 '01-2730': '2001-02-01',
 '01-1114': '2001-02-01',
 '01-1967': '2001-02-01',
 '01-1906': '2001-02-02',
 '01-2846': '2001-02-02',
 '01-2732': '2001-02-02',
 '01-2063': '2001-02-05',
 '01-2731': '2001-02-05',
 '01-2847': '2001-02-05',
 '01-3155': '2001-02-07',
 '01-3168': '2001-02-07',
 '01-3564': '2001-02-13',
 '01-3131': '2001-02-14',
 '01-4360': '2001-02-22',
 '01-4648': '2001-02-26',
 '01-4713': '2001-02-27',
 '01-5328': '2001-03-05',
 '01-5244': '2001-03-05',
 '01-5295': '2001-03-06',
 '01-5618': '2001-03-09',
 '01-5791': '2001-03-09',
 '01-5804': '2001-03-09',
 '01-6181': '2001-03-13',
 '01-6276': '2001-03-14',
 '01-6431': '2001-03-15',
 '01-6560': '2001-03-16',
 '01-6638': '2001-03-19',
 '01-6601': '2001-03-19',

In [37]:
files = glob("./docs/*")
files[0:5]

['./docs\\01-10222.xml',
 './docs\\01-10398.xml',
 './docs\\01-10407.xml',
 './docs\\01-1073.xml',
 './docs\\01-11005.xml']

In [39]:
basename = os.path.basename(files[0])
documentId = os.path.splitext(basename)[0]
documentId

'01-10222'

In [41]:
def preprocess_file(filename : str, MIN_LENGTH : int):
    infile = open(filename, encoding="utf8")
    contents = infile.read()
    soup = BeautifulSoup(contents, 'xml')
    p_tags = soup.find_all("P")
    
    # Filter out shortened phrases and then preprocess
    filter_short_phrases = [phrase.get_text() for phrase in p_tags if len(phrase.get_text()) > MIN_LENGTH]
    kw_mat = preprocess_documents(filter_short_phrases)
    
    # Flatters 2D List into 1D
    return [word for kws in kw_mat for word in kws]

In [47]:
files = glob("./docs/*")
print(len(files))

MIN_LENGTH = 300 # chosen parameter

for idx, file in enumerate(files):
    if idx % 500 == 0:
        print(idx)
    # Extract documentID from filepath
    basename = os.path.basename(file)
    documentId = os.path.splitext(basename)[0]
    if documentId not in id2date:
        print(documentId)
        continue
        
    # String of preprocessed words in given file
    words = preprocess_file(file, MIN_LENGTH)
    
#     np.save('{}_words.npy'.format(documentId), words)
    

3618
0
500
1000
1500
2000
2500
3000
3500


# Visualizations

In [7]:
from bs4 import BeautifulSoup
import numpy as np
import gensim
import glob
import os
from preprocessing import get_key_words
from gensim.models.ldamodel import LdaModel
import pyLDAvis
import pyLDAvis.gensim_models as gm

C:\Users\Rishi2\Anaconda3\lib\site-packages\sklearn\linear_model\_least_angle.py:34: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  method='lar', copy_X=True, eps=np.finfo(np.float).eps,
C:\Users\Rishi2\Anaconda3\lib\site-packages\sklearn\linear_model\_least_angle.py:164: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  method='lar', copy_X=T

In [6]:
path = r"C:\Junior Year\TAMUDatathon2021\docs" # Change this to the path of your /docs folder on your system

processed_docs = []

for filename in glob.glob(os.path.join(path, "*.xml")):
    processed_docs.append(get_key_words(filename))
    
dictionary = gensim.corpora.Dictionary(processed_docs)
dictionary.filter_extremes(no_below=5, no_above=0.5, keep_n= 100000)

In [8]:
num_topics = 5
corpus = [dictionary.doc2bow(text) for text in processed_docs]
lda_model = LdaModel(corpus, num_topics=num_topics)

C:\Users\Rishi2\Anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [9]:
vis = gm.prepare(topic_model=lda_model, corpus=corpus, dictionary=dictionary)
pyLDAvis.enable_notebook()
pyLDAvis.display(vis)

C:\Users\Rishi2\Anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
